<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Dataset processing
  </div> 
  
<div style="
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Clinical trials ICTRP
  </div>


  <div style=" float:left; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE -  Hybrid Intelligence
  </div> 
  
  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  December 2022
  </div>

<a id="TOC"></a>

#### Table Of Content

1. [Clinical trials](#texts) <br>


#### Useful links

- [Clinical Trials ICTRP dataset download](https://www.who.int/clinical-trials-registry-platform)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import re
import copy
import json
import zipfile

# data
import pandas as pd

# viz
# from tqdm import tnrange

#### Custom variables

In [3]:
path_to_data = os.path.join(os.getcwd(), 'clinical trials ICTRP')
path_to_data

'C:\\Users\\jb\\Desktop\\NLP\\Internal - Transformers for NLP\\datasets'

In [ ]:
base_dataset_name  = 'ICTRPFullExport-774632-24-03-2022.zip'
final_dataset_name = 'clinical-trials-ictrp.tsv'

<a id="texts"></a>

# 1. Clinical trials

[Table of content](#TOC)

In [42]:
def clean_text(s):
    s = (s if s else '')
    s = re.sub('<br>', '\n', s)
    s = re.sub('\n\n(\s)*-(\s)+', ', ', s)
    s = re.sub('(\s)+', ' ', s)
    s = (s[1: -1] if len(s.split(' '))>1 else s)
    s = s.strip()
    return s



def process_zipfile(zip_file):
    all_content = []
    archive = zipfile.ZipFile(zip_file, 'r')
    file = archive.namelist()[0]
    with archive.open(file, 'r') as f:
        return pd.read_csv(f)
    

    
def clean_dataframe(df):
    columns = [
        'public_title', 'Scientific_title',
        'Inclusion_Criteria', 'Exclusion_Criteria',
           'Primary_Outcome', 'Secondary_Outcomes',
    ]
    df = df[columns].fillna('').applymap(clean_text)
    df['Inclusion_Criteria'] = df.Inclusion_Criteria.apply(lambda s: re.sub('Inclusion Criteria(\s)*:(\s|,)*', '', s).strip())
    df['Exclusion_Criteria'] = df.Exclusion_Criteria.apply(lambda s: re.sub('Exclusion Criteria(\s)*:(\s|,)*', '', s).strip())
    return df

In [6]:
df = process_zipfile(os.path.join(path_to_data, base_dataset_name))

C:\Users\jb\AppData\Local\Temp\ipykernel_26060\2897665713.py:40: DtypeWarning: Columns (41,43,45,47,48,49,50,51,52,55,57,58,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(f)


In [7]:
df.shape

(774632, 63)

In [8]:
df.columns

Index(['TrialID', '(No column name)', 'SecondaryIDs', 'public_title',
       'Scientific_title', 'url', 'Public_Contact_Firstname',
       'Public_Contact_Lastname', 'Public_Contact_Address',
       'Public_Contact_Email', 'Public_Contact_Tel',
       'Public_Contact_Affiliation', 'Scientific_Contact_Firstname',
       'Scientific_Contact_Lastname', 'Scientific_Contact_Address',
       'Scientific_Contact_Email', 'Scientific_Contact_Tel',
       'Scientific_Contact_Affiliation', 'study_type', 'study_design', 'phase',
       'Date_registration', 'Date_enrollement', 'Target_size',
       'Recruitment_status', 'Primary_sponsor', 'Secondary_sponsors',
       'Source_Support', 'Countries', 'Conditions', 'Interventions', 'Age_min',
       'Age_max', 'Gender', 'Inclusion_Criteria', 'Exclusion_Criteria',
       'Primary_Outcome', 'Secondary_Outcomes', 'Bridging_flag',
       'Bridged_type', 'Childs', 'type_enrolment', 'Retrospective_flag',
       'results_actual_enrollment', 'results_url_link'

In [35]:
df = clean_dataframe(df)

In [50]:
df.head()

,TrialID,public_title,Scientific_title,Inclusion_Criteria,Exclusion_Criteria,Primary_Outcome,Secondary_Outcomes
0,T0504072,valuation of the Impact of the Administration ...,valuation of the Impact of the Administration ...,"patient = 18 years old,, symptomatic COVID-19 ...",,egativation of the RT-PCR test on nasopharynge...,
1,T0504075,orrelation Between Cardiac Markers and Severit...,orrelation Between Cardiac Markers and Severit...,all moderate to severe COVID 19 patients who w...,,elation between cardiac markers and mortality ...,elation ship between cardiac and inflammatory ...
2,T0504219,Prospective Virtual Study to Evaluate the Long...,Prospective Cohort Study of Immunoglobulin G (...,"Age = 18 years, Willing to provide informed co...",,o describe the longevity and seroprevalence of...,o compare the off-kinetics of IgG positivity a...
3,T0504223,Prospective Virtual Study of Patient Reported ...,"ssessing Behavioral, Functional, and Clinical ...","Age >=18 years, Individuals with a valid diagn...",,o identify and describe patient behaviors and ...,o describe recovery in patients with test-conf...
4,T0504454,Behavioral Activation Prenatal and Postpartum ...,ilot Study to Evaluate a Behavioral Activation...,"INCLUSION CRITERIA FOR AIMS 1, 3 AND 4, Pregna...",,ocus group,


In [57]:
df.to_csv(os.path.join(path_to_data, final_dataset_name), sep = "\t", index = False)

[Table of content](#TOC)